Python code for POS tagging. 
Implemented Viterbi algorithm 

In [1]:

def get_data(split_sequences=False):
    word2idx = {}  #Word Vocabulary with word as key and word_idx as value
    tag2idx = {}   #Tag Vocabulary with Tag as key and Tag_idx as value
    word_idx = 0   #counter for words
    tag_idx = 0    #counter for tags
    Xtrain = []    #Just for storing train sequences
    Ytrain = []    #Just for storing test sequences 
    currentX = []  #to store word sequences (observed Data)
    currentY = []  #to store tag sequences  (unobserved Data)
    for line in open('./data/pos/train.txt'):
        line = line.rstrip()
        if line:
            r = line.split()
            word, tag, _ = r
            if word not in word2idx:
                word2idx[word] = word_idx
                word_idx += 1
            currentX.append(word2idx[word])
            
            if tag not in tag2idx: 
                tag2idx[tag] = tag_idx
                tag_idx += 1
            currentY.append(tag2idx[tag])
        elif split_sequences:
            Xtrain.append(currentX)
            Ytrain.append(currentY)
            currentX = []
            currentY = []

    if not split_sequences:
        Xtrain = currentX
        Ytrain = currentY

    # load and score test data
    Xtest = []
    Ytest = []
    currentX = []
    currentY = []
    for line in open('./data/pos/test.txt'):
        line = line.rstrip()
        if line:
            r = line.split()
            word, tag, _ = r
            if word in word2idx:
                currentX.append(word2idx[word])
            else:
                currentX.append(word_idx) # use this as unknown
            currentY.append(tag2idx[tag])
        elif split_sequences:
            Xtest.append(currentX)
            Ytest.append(currentY)
            currentX = []
            currentY = []
    if not split_sequences:
        Xtest = currentX
        Ytest = currentY

    return Xtrain, Ytrain, Xtest, Ytest, word2idx

In [55]:

import numpy as np
import matplotlib.pyplot as plt

class HMM:
    def __init__(self, M):
        self.M = M # number of hidden states
    
#     def log_likelihood(self, x):
#         # returns log P(x | model)
#         # using the forward part of the forward-backward algorithm
#         T = len(x)
#         scale = np.zeros(T)
#         alpha = np.zeros((T, self.M))
#         alpha[0] = self.pi*self.B[:,x[0]]
#         scale[0] = alpha[0].sum()
#         alpha[0] /= scale[0]
#         for t in xrange(1, T):
#             alpha_t_prime = alpha[t-1].dot(self.A) * self.B[:, x[t]]
#             scale[t] = alpha_t_prime.sum()
#             alpha[t] = alpha_t_prime / scale[t]
#         return np.log(scale).sum()

#     def log_likelihood_multi(self, X):
#         return np.array([self.log_likelihood(x) for x in X])

    def get_state_sequence(self, x):
        # returns the most likely state sequence given observed sequence x
        # using the Viterbi algorithm
        T = len(x)
        delta = np.zeros((T, self.M))
        psi = np.zeros((T, self.M))
        delta[0] = np.log(self.pi) + np.log(self.B[:,x[0]])
        for t in xrange(1, T):
            for j in xrange(self.M):
                delta[t,j] = np.max(delta[t-1] + np.log(self.A[:,j])) + np.log(self.B[j, x[t]])
                psi[t,j] = np.argmax(delta[t-1] + np.log(self.A[:,j]))

        # backtrack
        states = np.zeros(T, dtype=np.int32)
        states[T-1] = np.argmax(delta[T-1])
        for t in xrange(T-2, -1, -1):
            states[t] = psi[t+1, states[t+1]]
        return states





In [56]:
import numpy as np
import matplotlib.pyplot as plt


from sklearn.utils import shuffle
from datetime import datetime
from sklearn.metrics import f1_score

def accuracy(T, Y):
    # inputs are lists of lists
    n_correct = 0
    n_total = 0
    for t, y in zip(T, Y):
        n_correct += np.sum(t == y)
        n_total += len(y)
    return float(n_correct) / n_total


def total_f1_score(T, Y):
    # inputs are lists of lists
    T = np.concatenate(T)
    Y = np.concatenate(Y)
    return f1_score(T, Y, average=None).mean()


# def flatten(l):
#     return [item for sublist in l for item in sublist]


def main(smoothing=10e-2):
    # X = words, Y = POS tags
    Xtrain, Ytrain, Xtest, Ytest, word2idx = get_data(split_sequences=True)
    V = len(word2idx) + 1

    # find hidden state, transition matrix and pi
    M = max(max(y) for y in Ytrain) + 1 #len(set(flatten(Ytrain))) (just to find total number of hiddent states)
    A = np.ones((M, M))*smoothing # add-one smoothing
    pi = np.zeros(M) 
    
    for y in Ytrain:
        pi[y[0]] += 1
        for i in xrange(len(y)-1):
            A[y[i], y[i+1]] += 1
            
    # turn it into a probability matrix
    A /= A.sum(axis=1, keepdims=True) #state transition matrix
    pi /= pi.sum() #emission matrix

    # find the observation matrix
    B = np.ones((M, V))*smoothing # add smoothing
    for x, y in zip(Xtrain, Ytrain):
        for xi, yi in zip(x, y):
            B[yi, xi] += 1
    B /= B.sum(axis=1, keepdims=True)

    hmm = HMM(M)
    hmm.pi = pi
    hmm.A = A
    hmm.B = B

    # get predictions
    Ptrain = []
    for x in Xtrain:
        p = hmm.get_state_sequence(x)
        Ptrain.append(p)

    Ptest = []
    for x in Xtest:
        p = hmm.get_state_sequence(x)
        Ptest.append(p)

    # print results
    print "train accuracy:", accuracy(Ytrain, Ptrain)
    print "test accuracy:", accuracy(Ytest, Ptest)
    print "train f1:", total_f1_score(Ytrain, Ptrain)
    print "test f1:", total_f1_score(Ytest, Ptest)

In [57]:
if __name__ == '__main__':
    main()

/Users/KrishMahajan/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:33: RuntimeWarning: divide by zero encountered in log


train accuracy: 0.973753937854
test accuracy: 0.928784009118
train f1: 0.923527954605
test f1: 0.862609207175


In [2]:
Xtrain, Ytrain, Xtest, Ytest, word2idx = get_data(split_sequences=True)

In [5]:
import numpy as np

In [6]:
M = max(max(y) for y in Ytrain) + 1 #len(set(flatten(Ytrain)))
A = np.ones((M, M))*(10e-2) # add-one smoothing
pi = np.zeros(M)

In [9]:
A.shape

(44, 44)

In [11]:
Ytrain[0]

[0,
 1,
 2,
 0,
 3,
 4,
 5,
 6,
 7,
 2,
 8,
 0,
 1,
 0,
 9,
 1,
 10,
 11,
 8,
 1,
 0,
 0,
 11,
 7,
 6,
 7,
 2,
 8,
 0,
 1,
 10,
 12,
 10,
 13,
 8,
 9,
 14]